# Alchemia danych: Jak Data Engineer tworzy wartość?
Witamy na webinarze infoShare Academy! Podczas tej sesji będziemy używać Pythona, aby zaprezentować kompletny przepływ pracy związany z przetwarzaniem danych. Oto, co omówimy:


## Agenda
1. Wprowadzenie do Pythona w przetwarzaniu danych
- Przegląd Pythona i jego ekosystemu do przetwarzania danych.
- Wprowadzenie do bibliotek, których będziemy używać: pandas i great_expectations.

2. Pobieranie danych z API Spotify
- Jak używać API Spotify do scrapowania danych.
- Pobieranie istotnych informacji, takich jak szczegóły utworów, nazwy artystów, popularność i gatunki muzyczne.

3. Prosty proces ETL (Extract, Transform, Load)
- Ekstrakcja danych z API Spotify.
- Transformacja danych do ustrukturyzowanego formatu za pomocą pandas.
- Ładowanie przekształconych danych do DataFrame.

4. Walidacja danych z Great Expectations
- Wprowadzenie do walidacji danych i jej znaczenia.
- Używanie great_expectations do walidacji danych.
- Ustawianie oczekiwań, aby zapewnić jakość danych.

5. Zapisywanie danych do CSV
- Eksportowanie zwalidowanych danych do pliku CSV.
- Upewnienie się, że dane są poprawnie zapisane do dalszej analizy lub udostępnienia.

## Narzędzia i biblioteki
- Python: Język programowania, którego będziemy używać do całego przepływu pracy.
- pandas: Potężna biblioteka do manipulacji danymi, która pomoże nam przekształcać i analizować dane.
- great_expectations: Biblioteka do walidacji danych, która zapewni, że nasze dane spełniają wymagane standardy jakości.
- Spotify API: Źródło naszych danych, z którego będziemy pobierać informacje o utworach i artystach.

## Wymagania wstępne

- Podstawowa znajomość programowania w Pythonie.
- Znajomość Jupyter Notebooks.
- Zrozumienie API i sposobu interakcji z nimi.

## Etap 1: Pobieranie danych z API Spotify
W pierwszym etapie naszego webinaru skupimy się na pobieraniu danych z API Spotify. API Spotify pozwala na dostęp do szerokiej gamy informacji o utworach, artystach, albumach i playlistach. Wykorzystamy te dane do dalszej analizy i przetwarzania.

Kroki do wykonania:
1. Rejestracja aplikacji na Spotify Developer Dashboard

- Aby uzyskać dostęp do API Spotify, musisz zarejestrować swoją aplikację na Spotify Developer Dashboard](https://developer.spotify.com/dashboard/applications).
- Po zarejestrowaniu aplikacji otrzymasz `Client ID` i `Client Secret`, które będą potrzebne do autoryzacji.

![alt text](images/spotify_api.png "Spotify API")

2. Upewnienie się, że zainstalowaliśmy bibliotekę `Spotipy` (lub instalacja biblioteki `Spotipy`)

- Spotipy to biblioteka Pythona, która ułatwia interakcję z API Spotify.
- Zainstaluj bibliotekę za pomocą poniższego polecenia:
```python
pip install spotipy
```

In [ ]:
import os
import json
import requests


def get_access_token(client_id: str, client_secret: str, grant_type: str = "client_credentials") -> str:
    """
    Retrieves an access token from the Spotify Accounts service.

    Parameters:
    client_id (str): The client ID provided by Spotify for your application.
    client_secret (str): The client secret provided by Spotify for your application.
    grant_type (str): The type of grant being requested. Default is "client_credentials".

    Returns:
    str: A string containing the access token prefixed with "Bearer ".

    Raises:
    requests.exceptions.RequestException: If the request to the Spotify Accounts service fails.
    KeyError: If the response does not contain an access token.

    Example:
    >>> get_access_token("your_client_id", "your_client_secret")
    'Bearer BQ...'

    Note:
    Ensure that you handle exceptions when calling this function, as network issues or invalid credentials can cause it to fail.
    """
    url = "https://accounts.spotify.com/api/token?grant_type={}&client_id={}&client_secret={}".format(grant_type, client_id, client_secret)
    response = requests.post(url, headers={'Content-Type':'application/x-www-form-urlencoded'})
    access_token = "Bearer " + json.loads(response.text)["access_token"]

    return access_token

In [ ]:
import os 

grant_type = "client_credentials"
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
access_token = get_access_token(client_id=client_id, client_secret=client_secret, grant_type=grant_type)

In [ ]:
client_id

In [ ]:
def get_data(url: str, access_token: str) -> dict:
    """
    Retrieves data from a specified URL using the provided access token for authorization.

    Parameters:
    url (str): The URL from which to retrieve data.
    access_token (str): The access token to be used for authorization in the request header.

    Returns:
    dict: A dictionary containing the JSON response from the URL.

    Raises:
    requests.exceptions.RequestException: If the request to the URL fails.
    json.JSONDecodeError: If the response is not valid JSON.
    ValueError: If the response does not contain the expected data.

    Example:
    >>> get_data("https://api.spotify.com/v1/me", "Bearer BQ...")
    {'id': 'user_id', 'display_name': 'User Name', ...}
    """
    try:
        response = requests.get(url, headers={"Authorization": access_token})
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        result = response.json()  # Parse JSON response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        raise
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON response: {e}")
        raise
    except ValueError as e:
        print(f"Unexpected data format: {e}")
        raise

    return result

In [ ]:
import pandas as pd 

def get_spotify_tracks(access_token: str, n_of_tracks: int, genres: list) -> pd.DataFrame:
    """
    Retrieves a specified number of tracks for each genre from the Spotify API and returns them as a DataFrame.

    Parameters:
    access_token (str): The access token to be used for authorization in the request header.
    n_of_tracks (int): The number of tracks to retrieve for each genre.
    genres (list): A list of genres for which to retrieve tracks.

    Returns:
    pd.DataFrame: A DataFrame containing the retrieved tracks with the following columns:
        - track_id: The ID of the track.
        - track_name: The name of the track.
        - genre: The genre of the track.
        - artist_name: The name of the artist.
        - popularity: The popularity score of the track.
        - duration_ms: The duration of the track in milliseconds.
        - album_name: The name of the album.
        - total_tracks_in_album: The total number of tracks in the album.

    Raises:
    requests.exceptions.RequestException: If the request to the Spotify API fails.
    json.JSONDecodeError: If the response is not valid JSON.
    ValueError: If the response does not contain the expected data.

    Example:
    >>> get_spotify_tracks("Bearer BQ...", 10, ["rock", "pop"])
    DataFrame with columns: track_id, track_name, genre, artist_name, popularity, duration_ms, album_name, total_tracks_in_album
    """
    spotify_tracks = []

    for genre in genres:
        url = f"https://api.spotify.com/v1/search?q=genre:{genre}&type=track&limit={n_of_tracks}&sort=popularity.desc"
        items = get_data(url=url, access_token=access_token)
        
        for track in items["tracks"]["items"]:
            track_info = {
                "track_id": track["id"],
                "track_name": track["name"],
                "genre": genre,
                "artist_name": track["artists"][0]["name"],
                "popularity": track["popularity"],
                "duration_ms": track["duration_ms"],
                "album_name": track["album"]["name"],
                "total_tracks_in_album": track["album"]["total_tracks"],
            }
            spotify_tracks.append(track_info)

    spotify_tracks_df = pd.DataFrame(spotify_tracks)
    return spotify_tracks_df

In [ ]:
genres = ["techno", "pop", "rock", "jazz", "classical"]

spotify_tracks_df = get_spotify_tracks(access_token=access_token, n_of_tracks=50, genres=genres)

In [ ]:
spotify_tracks_df

Możemy za pomocą API dostać dużo ciekawych informacji np.: 

![alt text](images/audio_analysis.png "Audio analysis API")

## Etap 2: Transformacje

### Transformacja danych obejmuje kilka kroków:

- Grupowanie po gatunku: Dane są grupowane według gatunku muzycznego.
- Obliczanie średniej popularności: Dla każdego gatunku obliczamy średnią wartość popularności utworów.
- Obliczanie średniej długości utworów: Dla każdego gatunku obliczamy średnią długość utworów w milisekundach.
- Obliczanie liczby utworów na albumie: Dla każdego albumu obliczamy liczbę utworów, a następnie obliczamy średnią liczbę utworów na albumie dla każdego gatunku.


In [ ]:
# Group by genre and calculate avg values
df_after_transformation = spotify_tracks_df.groupby("genre").agg({
  "popularity": "mean",
  "duration_ms": "mean",
  "total_tracks_in_album": "mean",
}).reset_index()

In [ ]:
df_after_transformation

In [ ]:
# Rename columns for clarity
df_after_transformation.columns = ["genre", "avg_popularity", "avg_duration_ms", "avg_total_tracks_in_album"]

df_after_transformation

In [ ]:
def format_duration(duration_ms: int) -> str:
    minutes = duration_ms // 60000
    seconds = (duration_ms % 60000) // 1000
    return f"{int(minutes)} min {int(seconds)} sec"

In [ ]:
# Calculate duration in format min and sec
df_after_transformation["avg_duration_ms"] = df_after_transformation["avg_duration_ms"].apply(lambda duration_ms: format_duration(duration_ms))
df_after_transformation

In [ ]:
df_after_transformation.columns = ["genre", "avg_popularity", "avg_duration", "avg_total_tracks_in_album"]
df_after_transformation

In [ ]:
df = df_after_transformation

## Etap 3: Walidacja danych za pomocą Great Expectations

W tym module będziemy używać biblioteki `Great Expectations` do walidacji danych zebranych z API Spotify. Naszym celem jest upewnienie się, że:
- Kolumna genre zawiera tylko wartości z określonej listy gatunków muzycznych.
- Wartości w kolumnie popularity mieszczą się w przedziale od 0 do 100.

Link do biblioteki --> https://greatexpectations.io/ (w tym module wykorzystujemy wersję `0.18.21`)

In [ ]:
import great_expectations

# Define the list of valid genres
valid_genres = ["techno", "pop", "rock", "jazz", "classical"]

In [ ]:
ge_df = great_expectations.from_pandas(df_after_transformation)

In [ ]:
# Check if genre is in the list of valid genres
genre_expectation = ge_df.expect_column_values_to_be_in_set("genre", valid_genres)
genre_expectation

In [ ]:
# Check if avg_popularity is between 0 and 100
popularity_expectation = ge_df.expect_column_values_to_be_between("avg_popularity", 0, 100)
popularity_expectation


## Etap 4: Zapisanie danych do pliku CSV

W tym etapie zapiszemy przetworzone i zweryfikowane dane do pliku CSV. Dzięki temu będziemy mogli łatwo udostępniać dane lub używać ich w innych aplikacjach.

Kroki:
- Przetworzenie danych: Upewnimy się, że dane są przetworzone i zweryfikowane zgodnie z naszymi oczekiwaniami.
- Zapisanie do pliku CSV: Użyjemy funkcji to_csv z biblioteki pandas, aby zapisać dane do pliku CSV.

In [ ]:
# Zapisz DataFrame do pliku CSV

df.to_csv("spotify_summary.csv", index=False)

Koniec tej części demo :) 